In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

### Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load transfection data from exp83.5 (iPS11 cells)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp83_3_path = base_path/'2024.04.11_exp83.3'/'export'
exp83_5_path = base_path/'2024.04.29_exp83.5'/'export'

plates = pd.DataFrame({
    'data_path': [exp83_3_path/'plate1_biorep1', exp83_3_path/'plate1_biorep2', exp83_3_path/'plate2_biorep3',
                  exp83_5_path],
    
    'yaml_path': [exp83_3_path/'exp83.3_biorep1_wells.yaml', exp83_3_path/'exp83.3_biorep2_wells.yaml', exp83_3_path/'exp83.3_biorep1_wells.yaml', 
                  exp83_5_path/'exp83.5_wells.yaml'],
    
    'biorep': [1, 2, 3,
               4],
    
    'exp': ['exp83.3']*3 + ['exp83.5']
})

output_path = rd.rootdir/'output'/'iPSC'/'BATS'
cache_path = rd.rootdir/'output'/'iPSC'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp','biorep'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
#gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
#gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
full_name = {'NT': 'uregulated', 'T': 'circuit'}

# Create new color palette for comparing promoters
metadata2 = metadata.copy()
metadata2.loc[(metadata2['promoter']=='CAG') & (metadata2['ts_kind']=='T') & (metadata2['group']=='controller'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['promoter']=='CAG') & (metadata2['ts_kind']=='NT'), 'color'] = metadata2.loc[(metadata2['promoter']=='CAG') & (metadata2['ts_kind']=='NT'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['promoter']=='CAG') & (metadata2['ts_kind']=='NT'), 'markers'] = 'P'
metadata2.loc[(metadata2['promoter']=='CAG') & (metadata2['ts_kind']=='T'), 'markers'] = 'D'
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
promoter_palette = metadata_dict2['color']
promoter_markers = metadata_dict2['markers']

In [ ]:
# Line plot with marginal histogram, EF1a
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['promoter']=='EF1a') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==3)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,2e5), ylim=(4e0,1e6),)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==3), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['promoter']=='EF1a') & (stats['group']=='controller') & (stats['design']==1) & (stats['biorep']<4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_EF1a_biorep1-3.svg')), bbox_inches='tight')

In [ ]:
# Line plot with marginal histogram, EF1a
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['promoter']=='EF1a') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,2e5), ylim=(4e0,1e6),) 
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['promoter']=='EF1a') & (stats['group']=='controller') & (stats['design']==1) & (stats['biorep']==4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_EF1a_biorep4.svg')), bbox_inches='tight')

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==3)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,2e5), ylim=(4e0,1e6),) 
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==3), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']<4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_CAG_biorep1-3.svg')), bbox_inches='tight')

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,2e5), ylim=(4e0,1e6),)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']==4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_CAG_biorep4.svg')), bbox_inches='tight')

In [ ]:
# Bin by marker quantiles - combine bioreps 1-3
by = ['construct','exp']
df2 = df.copy()
df2['bin_marker_quantiles'] = df2.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df2.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles2 = df2.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles2)

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles2[(df_quantiles2['group']=='controller') & (df_quantiles2['design']==1) & (df_quantiles2['biorep']<4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,2e5), ylim=(4e0,1e6),) 
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']<4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']<4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_CAG_biorep1-3-combined.svg')), bbox_inches='tight')

### Figures for BATS

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(1e3,2e5), ylim=(5e1,1e6),)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']==4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_CAG_biorep4_BATS.svg')), bbox_inches='tight')

In [ ]:
# Line plot with marginal histogram, EF1a
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==4) & (df_quantiles['promoter']=='EF1a')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(1e3,2e5), ylim=(5e1,1e6),)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']==4) & (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_EF1a_biorep4_BATS.svg')), bbox_inches='tight')

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(((df_quantiles['group']=='controller') & (df_quantiles['design']==1)) | (df_quantiles['group']=='marker')) & (df_quantiles['biorep']==4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(1e3,2e5), )#ylim=(2e2,1e6),)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['group']=='controller') & (stats['design']==1) & (stats['biorep']==4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 3.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_CAG_biorep4_BATS-line.svg')), bbox_inches='tight')

In [ ]:
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

In [ ]:
stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

In [ ]:
# Line plot with marginal histogram, CAG
fig, axes = plt.subplots(1,3, figsize=(9,5), gridspec_kw=dict(width_ratios=(1,0.3,0.3), wspace=0.35))
plot_df = df_quantiles[(df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['biorep']==4)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=promoter_palette, 
             legend=False, dashes=False, style='construct', markers=promoter_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=(1e3,2e5), ylim=(5e1,1e6), xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==4), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=promoter_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=(5e1,1e6), yticklabels=['']*(len(ax.get_yticklabels())-1), ylabel='')

# slope
ax = axes[2]
plot_df = fits[(fits['group']=='controller') & (fits['design']==1) & (fits['biorep']==4)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=promoter_palette, 
                    legend=False, ax=ax, marker=promoter_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(1.1e3), np.log10(1.4e5), 1000)
    axes[0].plot(10**(x_range), 10**(group['slope'].values[0] * x_range + group['intercept_log'].values[0]), color='black', zorder=100)

sns.despine(ax=ax)
ax.set(title='Slope', xlim=(-0.5, 3.5), xlabel='', ylabel='', ylim=(0,1.05))
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-slope_CAG_biorep4_BATS.svg')), bbox_inches='tight')